# Reducing Failure-Inducing Inputs

By construction, fuzzers create inputs that may be hard to read.  This causes issues during _debugging_, when a human has to analyze the exact cause of the failure.  In this chapter, we present techniques that _automatically reduce and simplify failure-inducing inputs to a minimum_ in order to ease debugging.

**Prerequisites**

* The simple "delta debugging" technique for reduction has no specific prerequisites.

This chapter is adapted from [a similar chapter in "The Fuzzing Book"](https://www.fuzzingbook.org/html/Reducer.html). The interface has been simplified to be independent from the `fuzzingbook` infrastructure.

## Synopsis
<!-- Automatically generated. Do not edit. -->

To [use the code provided in this chapter](Importing.ipynb), write

```python
>>> from debuggingbook.DeltaDebugger import <identifier>
```

and then make use of the following features.


A _reducer_ takes a failure-inducing input and reduces it to the minimum that still reproduces the failure.  This chapter provides a `DeltaDebugger` class that implements such a reducer.

Here is a simple example: An arithmetic expression causes an error in the Python interpreter:

```python
>>> def myeval(inp):
>>>     return eval(inp)
>>> with ExpectError():
>>>     myeval('1 + 2 * 3 / 0')
Traceback (most recent call last):
  File "<ipython-input-80-deae668eac94>", line 2, in <module>
    myeval('1 + 2 * 3 / 0')
  File "<ipython-input-79-e6018665ac61>", line 2, in myeval
    return eval(inp)
  File "<string>", line 1, in <module>
ZeroDivisionError: division by zero (expected)
```
Can we reduce this input to a minimum? _Delta Debugging_ is a simple and robust reduction algorithm. We provide a `DeltaDebugger` class that is used in conjunction with a (failing) function call:

```python
with DeltaDebugger():
    fun(args...)
```

The class automatically determines minimal arguments that cause the function to fail with the same exception as the original. By default, these minimal arguments are simply printed out:

```python
>>> with DeltaDebugger():
>>>     myeval('1 + 2 * 3 / 0')
myeval(inp='2/0')
```
The input is reduced to the maximum: We get the essence of the division by zero.

There also is an interface to access the reduced input(s) programmatically:

```python
>>> dd = DeltaDebugger(show=False)
>>> with dd:
>>>     myeval('1 + 2 * 3 / 0')
>>> dd.reduced_args()
{'inp': '2/0'}
```
`DelteDebugger` works on all functions that take a collection (a string, a list, a set) as one or more of their inputs.



## Why Reducing?

A common problem in debugging is that given an input, only a _small part of that input may be responsible for the failure_. A central part of debugging is to _identify_ these parts – and to simplify (or _reduce_) the input to a minimal form that reproduces the failure – but does and contains as little else as possible.

Here's an example of such a situation.  We have a `mystery()` method that – given its code – can occasionally fail.  But under which circumstances does this actually happen?  We have deliberately obscured the exact condition in order to make this non-obvious.

In [1]:
import bookutils

In [2]:
from bookutils import quiz

In [3]:
import re

In [4]:
def mystery(inp):
    x = inp.find(chr(0o17 + 0o31))
    y = inp.find(chr(0o27 + 0o22))
    if x >= 0 and y >= 0 and x < y:
        raise ValueError("Invalid input")
    else:
        pass

Let us _fuzz_ the function – that is, feed it with random inputs – until we find a failing input. We introduce a simple `fuzz()` function for this purpose:

The function `random.randrange(a, b)` returns a random number in the range (a, b).

In [5]:
import random

In [6]:
random.randrange(32, 128)

107

We can use `random.randrange()` to compose random (printable) characters:

In [7]:
def fuzz():
    length = random.randrange(10, 70)
    fuzz = ""
    for i in range(length):
        fuzz += chr(random.randrange(32, 127))
    return fuzz

Here are some random strings produced by our `fuzz()` function:

In [8]:
for i in range(6):
    print(repr(fuzz()))

'N&+slk%hyp5'
"'@[3(rW*M5W]tMFPU4\\P@tz%[X?uo\\1?b4T;1bDeYtHx #UJ5"
'w}pMmPodJM,_%%BC~dYN6*g|Y*Ou9I<P94}7,99ivb(9`=%jJj*Y*d~OLXk!;J'
"!iOU8]hqg00?u(c);>:\\=V<ZV1=*g#UJA'No5QZ)~--[})Sdv#m*L"
'0iHh[-MzS.U.X}fG7aA:G<bEI\'Ofn[",Mx{@jfto}i3D?7%V7XdtO6BjYEa#Il)~]'
"E`h7h)ChX0G*m,|sosJ.mu/\\c'EpaPi0(n{"


Let us now use `fuzz()` to find an input where `mistery()` fails:

In [9]:
while True:
    inp = fuzz()
    try:
        mystery(inp)
    except ValueError:
        break

This is an input that causes `mystery()` to fail:

In [10]:
failing_input = inp
failing_input

'V"/+!aF-(V4EOz*+s/Q,7)2@0_'

In [11]:
from ExpectError import ExpectError

In [12]:
with ExpectError():
    mystery(failing_input)

Traceback (most recent call last):
  File "<ipython-input-12-2c6d37965be6>", line 2, in <module>
    mystery(failing_input)
  File "<ipython-input-4-26b70a4c8761>", line 5, in mystery
    raise ValueError("Invalid input")
ValueError: Invalid input (expected)


Something in this input causes `MysteryRunner` to fail.  But what is it?

## Manual Input Reduction

One important step in the debugging process is _reduction_ – that is, to identify those circumstances of a failure that are relevant for the failure to occur, and to _omit_ (if possible) those parts that are not.  As Kernighan and Pike \cite{Kernighan1999} put it:

> For every circumstance of the problem, check whether it is relevant for the problem to occur.  If it is not, remove it from the problem report or the test case in question.

Specifically for inputs, they suggest a _divide and conquer_ process:

> Proceed by binary search.  Throw away half the input and see if the output is still wrong; if not, go back to the previous state and discard the other half of the input.

This is something we can easily try out, using our last generated input:

In [13]:
failing_input

'V"/+!aF-(V4EOz*+s/Q,7)2@0_'

For instance, we can see whether the error still occurs if we only feed in the first half:

In [14]:
half_length = len(failing_input) // 2   # // is integer division
first_half = failing_input[:half_length]
first_half

'V"/+!aF-(V4EO'

In [15]:
with ExpectError():
    mystery(first_half)

Nope – the first half alone does not suffice.  Maybe the second half?

In [16]:
second_half = failing_input[half_length:]
assert first_half + second_half == failing_input
second_half

'z*+s/Q,7)2@0_'

In [17]:
with ExpectError():
    mystery(second_half)

This did not go so well either.  We may still proceed by cutting away _smaller chunks_ – say, one character after another.  If our test is deterministic and easily repeated, it is clear that this process eventually will yield a reduced input.  But still, it is a rather inefficient process, especially for long inputs.  What we need is a _strategy_ that effectively minimizes a failure-inducing input – a strategy that can be automated.

## Delta Debugging

One strategy to effectively reduce failure-inducing inputs is _delta debugging_ \cite{Zeller2002}.  Delta Debugging implements the "binary search" strategy, as listed above, but with a twist: If neither half fails (also as above), it keeps on cutting away smaller and smaller chunks from the input, until it eliminates individual characters.  Thus, after cutting away the first half, we cut away
the first quarter, the second quarter, and so on.

Let us illustrate this on our example, and see what happens if we cut away the first quarter.

In [18]:
quarter_length = len(failing_input) // 4
input_without_first_quarter = failing_input[quarter_length:]
input_without_first_quarter

'F-(V4EOz*+s/Q,7)2@0_'

In [19]:
with ExpectError():
    mystery(input_without_first_quarter)

Traceback (most recent call last):
  File "<ipython-input-19-3405be4be6b3>", line 2, in <module>
    mystery(input_without_first_quarter)
  File "<ipython-input-4-26b70a4c8761>", line 5, in mystery
    raise ValueError("Invalid input")
ValueError: Invalid input (expected)


Ah! This has failed, and reduced our failing input by 25%.  Let's remove another quarter.

In [20]:
input_without_first_and_second_quarter = failing_input[quarter_length * 2:]
input_without_first_and_second_quarter

'Oz*+s/Q,7)2@0_'

In [21]:
with ExpectError():
    mystery(input_without_first_and_second_quarter)

This is not too surprising, as we had that one before:

In [22]:
second_half

'z*+s/Q,7)2@0_'

In [23]:
input_without_first_and_second_quarter

'Oz*+s/Q,7)2@0_'

How about removing the third quarter, then?

In [24]:
input_without_first_and_third_quarter = failing_input[quarter_length:
                                                      quarter_length * 2] + failing_input[quarter_length * 3:]
input_without_first_and_third_quarter

'F-(V4EQ,7)2@0_'

In [25]:
with ExpectError():
    mystery(input_without_first_and_third_quarter)

Traceback (most recent call last):
  File "<ipython-input-25-2736f29f5713>", line 2, in <module>
    mystery(input_without_first_and_third_quarter)
  File "<ipython-input-4-26b70a4c8761>", line 5, in mystery
    raise ValueError("Invalid input")
ValueError: Invalid input (expected)


Yes!  This has succeeded.  Our input is now 50% smaller.

We have now tried to remove pieces that make up $\frac{1}{2}$ and $\frac{1}{4}$ of the original failing string.  In the next iteration, we would go and remove even smaller pieces – $\frac{1}{8}$, $\frac{1}{16}$ and so on.  We continue until we are down to $\frac{1}{26}$ – that is, individual characters.

However, this is something we happily let a computer do for us – and this is what the _Delta Debugging_ algorithm does.  Delta Debugging implements the strategy sketched above: It first removes larger chunks of size $\frac{1}{2}$; if this does not fail, then we proceed to chunks of size $\frac{1}{4}$, then $\frac{1}{8}$ and so on.

Our `ddmin()` implementation uses almost the same Python code as Zeller in \cite{Zeller2002}; the only difference is that it has been adapted to work on Python 3.  The variable `n` (initially 2) indicates the granularity – in each step, chunks of size $\frac{1}{n}$ are cut away.  If none of the test fails (`some_complement_is_failing` is False), then `n` is doubled – until it reaches the length of the input.

In [26]:
PASS = 'PASS'
FAIL = 'FAIL'
UNRESOLVED = 'UNRESOLVED'

In [27]:
def ddmin(test, inp, *test_args):
    """Reduce the input inp, using the outcome of test(fun, inp)."""
    assert test(inp, *test_args) != PASS

    n = 2     # Initial granularity
    while len(inp) >= 2:
        start = 0
        subset_length = len(inp) / n
        some_complement_is_failing = False

        while start < len(inp):
            complement = inp[:int(start)] + \
                inp[int(start + subset_length):]

            if test(complement, *test_args) == FAIL:
                inp = complement
                n = max(n - 1, 2)
                some_complement_is_failing = True
                break

            start += subset_length

        if not some_complement_is_failing:
            if n == len(inp):
                break
            n = min(n * 2, len(inp))

    return inp

To see how `reduce()` works, let us run it on our failing input. We need to define a `test` function that returns PASS or FAIL, depending on the test outcome. This `generic_test()` assumes that the function fails if it raises an exception (such as an `AssertException`), and passes otherwise. The optional argument `expected_exc` specifies the name of exception to be checked for; this ensures we reduce only for the kind of error raised in the original failure.

In [28]:
def generic_test(inp, fun, expected_exc=None):
    result = None
    detail = ""
    try:
        result = fun(inp)
        outcome = PASS
    except Exception as exc:
        detail = f" ({type(exc).__name__}: {str(exc)})"
        if expected_exc is None:
            outcome = FAIL
        elif type(exc) == type(expected_exc) and str(exc) == str(expected_exc):
            outcome = FAIL
        else:
            outcome = UNRESOLVED

    print(f"{fun.__name__}({repr(inp)}): {outcome}{detail}")
    return outcome

We can now invoke `ddmin()` in our setting. With each step, we see how the remaining input gets smaller and smaller, until only two characters remain:

In [29]:
ddmin(generic_test, failing_input, mystery, ValueError('Invalid input'))

mystery('V"/+!aF-(V4EOz*+s/Q,7)2@0_'): FAIL (ValueError: Invalid input)
mystery('z*+s/Q,7)2@0_'): PASS
mystery('V"/+!aF-(V4EO'): PASS
mystery('F-(V4EOz*+s/Q,7)2@0_'): FAIL (ValueError: Invalid input)
mystery('Oz*+s/Q,7)2@0_'): PASS
mystery('F-(V4E,7)2@0_'): FAIL (ValueError: Invalid input)
mystery(',7)2@0_'): PASS
mystery('F-(V4E'): PASS
mystery('V4E,7)2@0_'): PASS
mystery('F-(,7)2@0_'): FAIL (ValueError: Invalid input)
mystery(',7)2@0_'): PASS
mystery('F-(2@0_'): PASS
mystery('F-(,7)'): FAIL (ValueError: Invalid input)
mystery(',7)'): PASS
mystery('F-('): PASS
mystery('-(,7)'): FAIL (ValueError: Invalid input)
mystery('(,7)'): FAIL (ValueError: Invalid input)
mystery('7)'): PASS
mystery('(,'): PASS
mystery(',7)'): PASS
mystery('(7)'): FAIL (ValueError: Invalid input)
mystery('7)'): PASS
mystery('()'): FAIL (ValueError: Invalid input)
mystery(')'): PASS
mystery('('): PASS


'()'

Now we know why `MysteryRunner` fails – it suffices that the input contains two matching parentheses.  Delta Debugging determines this in 32 steps.  Its result is _1-minimal_, meaning that every character contained is required to produce the error; removing any (as seen in the last two tests, above) no longer makes the test fail.  This property is guaranteed by the delta debugging algorithm, which in its last stage always tries to delete characters one by one.

A reduced test case such as the one above has many advantages:

* A reduced test case __reduces the _cognitive load_ of the programmer__.  The test case is shorter and focused, and thus does not burden the programmer with irrelevant details.  A reduced input typically leads to shorter executions and smaller program states, both of which reduce the search space as it comes to understanding the bug.  In our case, we have eliminated lots of irrelevant input – only the two characters the reduced input contains are relevant.

* A reduced test case __is easier to communicate__.  All one needs here is the summary: `MysteryRunner fails on "()"`, which is much better than `MysteryRunner fails on a 4100-character input (attached)`.

* A reduced test case helps in __identifying duplicates__.  If similar bugs have been reported already, and all of them have been reduced to the same cause (namely that the input contains matching parentheses), then it becomes obvious that all these bugs are different symptoms of the same underlying cause – and would all be resolved at once with one code fix.

How effective is delta debugging?  In the best case (when the left half or the right half fails), the number of tests is logarithmic proportional to the length $n$ of an input (i.e., $O(\log_2 n)$); this is the same complexity as binary search.  In the worst case, though, delta debugging can require a number of tests proportional to $n^2$  (i.e., $O(n^2)$) – this happens in the case when we are down to character granularity, and we have to repeatedly tried to delete all characters, only to find that deleting the last character results in a failure \cite{Zeller2002}.  (This is a pretty pathological situation, though.)

In general, delta debugging is a robust algorithm that is easy to implement, easy to deploy, and easy to use – provided that the underlying test case is deterministic and runs quickly enough to warrant a number of experiments. In general, any debugging task should start with simplifying the test case as much as possible – and this is where delta debugging can help.

## A Simple Interface

As defined above, using `ddmin()` still requires the developer to set up a special testing function. We want to simplify the setup such that only a single Python line is required.

Our aim is to have a `DeltaDebugger` class that we can use in conjunction with a failing (i.e., exception raising) function call:

```python
with DeltaDebugger():
    mystery(failing_input)
```
Here, at the end of the `with` statement, the debugger would print out the minimal input that causes the failure.

### Collecting a Call

In [30]:
import sys

In [31]:
from types import FunctionType

In [32]:
class CallCollector(object):
    def __init__(self):
        """Reduce a function call."""
        self._function = None
        self._args = None
        self._exception = None

    def traceit(self, frame, event, arg):
        """Tracing function. Collect first call."""
        if event == 'call':
            name = frame.f_code.co_name
            if name.startswith('__'):
                # Internal function
                return

            self._function = FunctionType(frame.f_code,
                                          globals=globals(),
                                          name=name)
            self._args = frame.f_locals

            # Turn tracing off
            sys.settrace(self.original_trace_function)

    def diagnosis(self):
        """Produce a diagnosis. To be defined in subclasses."""
        pass

    def args(self):
        """Return the dictionary of collected arguments."""
        return self._args

    def function(self):
        """Return the function called."""
        return self._function

    def __enter__(self):
        """Called at begin of `with` block. Turn tracing on."""
        self.original_trace_function = sys.gettrace()
        sys.settrace(self.traceit)

    def __exit__(self, exc_type, exc_value, traceback):
        """Called at end of `with` block. Turn tracing off."""
        sys.settrace(self.original_trace_function)
        if exc_type is not None and self._function is None:
            raise exc_value

        self._exception = exc_value
        self.diagnosis()
        return True  # Ignore exception

In [33]:
call_collector = CallCollector()
with call_collector:
    mystery(failing_input)

In [34]:
call_collector._function

<function __main__.mystery(inp)>

In [35]:
call_collector._args

{'inp': 'V"/+!aF-(V4EOz*+s/Q,7)2@0_'}

In [36]:
call_collector._exception

ValueError('Invalid input')

How robust is this?

In [37]:
with ExpectError():
    c = CallCollector()
    with c:
        some_error()

Traceback (most recent call last):
  File "<ipython-input-37-c9d15de4afd4>", line 4, in <module>
    some_error()
  File "<ipython-input-32-ece4b5584fab>", line 45, in __exit__
    raise exc_value
  File "<ipython-input-37-c9d15de4afd4>", line 4, in <module>
    some_error()
NameError: name 'some_error' is not defined (expected)


### Repeating a Call

In [38]:
with ExpectError():
    call_collector._function("foo")

In [39]:
with ExpectError():
    call_collector._function(failing_input)

Traceback (most recent call last):
  File "<ipython-input-39-8b3b221fbbf8>", line 2, in <module>
    call_collector._function(failing_input)
  File "<ipython-input-4-26b70a4c8761>", line 5, in mystery
    raise ValueError("Invalid input")
ValueError: Invalid input (expected)


In [40]:
with ExpectError():
    call_collector._function(**call_collector._args)

Traceback (most recent call last):
  File "<ipython-input-40-7f654363de6a>", line 2, in <module>
    call_collector._function(**call_collector._args)
  File "<ipython-input-4-26b70a4c8761>", line 5, in mystery
    raise ValueError("Invalid input")
ValueError: Invalid input (expected)


In [41]:
class CallCollector(CallCollector):
    def call(self, new_args={}):
        args = {}
        for var in self._args:
            args[var] = self._args[var]
        for var in new_args:
            args[var] = new_args[var]

        return self._function(**new_args)

In [42]:
call_collector = CallCollector()
with call_collector:
    mystery(failing_input)

In [43]:
with ExpectError():
    call_collector.call({'inp': 'foo'})

### Reducing Inputs

We first introduce a `Reducer` class as an abstract superclass for all kinds of reducers.

The `test()` method runs a single test (with logging, if wanted); the `reduce()` method will eventually reduce an input to the minimum.

In [44]:
class Reducer(CallCollector):
    def __init__(self, log=False):
        super().__init__()
        self.log = log
        self.reset()

    def reset(self):
        self.tests = 0

    def run(self, args):
        try:
            result = self.call(args)
        except Exception as exc:
            self.last_exception = exc
            if type(exc) == type(self._exception) and str(exc) == str(self._exception):
                return FAIL
            else:
                return UNRESOLVED  # Some other failure

        self.last_result = result
        return PASS

    def format_call(self, args=None):
        if args is None:
            args = self._args
        return self._function.__name__ + "(" + \
            ", ".join(f"{arg}={repr(args[arg])}" for arg in args) + ")"

    def test(self, args):
        outcome = self.run(args)
        if outcome == PASS:
            detail = ""
        else:
            detail = type(self.last_exception).__name__
            if str(self.last_exception):
                detail += ": " + str(self.last_exception)
            detail = f" ({detail})"

        self.tests += 1
        if self.log:
            print(f"Test #{self.tests} {self.format_call(args)}: {outcome}{detail}")

        return outcome

The `CachingReducer` variant saves test results, such that we don't have to run the same tests again and again:

In [45]:
class CachingReducer(Reducer):
    def reset(self):
        super().reset()
        self.cache = {}

    def test(self, args):
        index = ((k, v) for k, v in args.items())
        if index in self.cache:
            return self.cache[index]

        outcome = super().test(args)
        self.cache[index] = outcome
        return outcome

Here comes the _Delta Debugging_ reducer.  Delta Debugging implements the strategy sketched above: It first removes larger chunks of size $\frac{1}{2}$; if this does not fail, then we proceed to chunks of size $\frac{1}{4}$, then $\frac{1}{8}$ and so on.

In [46]:
class DeltaDebugger(CachingReducer):
    def __init__(self, show=True, **args):
        super().__init__(**args)
        self.show_diagnosis = show
        self._reduced_args = None

    def reduce(self, var_to_be_reduced, args):
        self.reset()
        assert self.test(args) != PASS, f"{self.format_call(args)} did not pass"
        inp = args[var_to_be_reduced]

        n = 2     # Initial granularity
        while len(inp) >= 2:
            start = 0
            subset_length = len(inp) / n
            some_complement_is_failing = False

            while start < len(inp):
                complement = inp[:int(start)] + \
                    inp[int(start + subset_length):]

                new_args = {}
                for var in args:
                    new_args[var] = args[var]
                new_args[var_to_be_reduced] = complement
                if self.test(new_args) == FAIL:
                    inp = complement
                    n = max(n - 1, 2)
                    some_complement_is_failing = True
                    break

                start += subset_length

            if not some_complement_is_failing:
                if n == len(inp):
                    break
                n = min(n * 2, len(inp))

        return inp

In [47]:
class DeltaDebugger(DeltaDebugger):
    def reducible(self, arg):
        try:
            x = len(arg)
        except TypeError:
            return False
        
        try:
            x = arg[0]
        except TypeError:
            return False
        except IndexError:
            return False
        
        return True

In [48]:
class DeltaDebugger(DeltaDebugger):
    def reduced_args(self):
        if self._reduced_args is not None:
            return self._reduced_args

        args = {}
        for var in self._args:
            args[var] = self._args[var]
        vars_to_be_reduced = set(args.keys())
        
        while len(vars_to_be_reduced) > 0:
            for var in vars_to_be_reduced:
                value = args[var]
                if not self.reducible(value):
                    vars_to_be_reduced.remove(var)
                    break
                if self.log:
                    print(f"Reducing {var}...")
                reduced_value = self.reduce(var, args)
                if len(reduced_value) < len(value):
                    args[var] = reduced_value
                    vars_to_be_reduced = set(args.keys())
                vars_to_be_reduced.remove(var)
                break

        assert self.test(args) == FAIL, f"{self.format_call(args)} does not fail"
        self._reduced_args = args
        return args

In [49]:
class DeltaDebugger(DeltaDebugger):
    def diagnosis(self):
        if self._function is None:
            raise ValueError("No function call observed")
        if self._exception is None:
            raise ValueError(f"{self.format_call()} did not raise an exception")

        reduced_args = self.reduced_args()
        if self.show_diagnosis:
            print(self.format_call(reduced_args))
        return reduced_args

To see how the `DeltaDebuggingReducer` works, let us run it on our failing input.  With each step, we see how the remaining input gets smaller and smaller, until only two characters remain:

In [50]:
with DeltaDebugger():
    mystery(failing_input)

mystery(inp='()')


In [51]:
with DeltaDebugger(log=True):
    mystery(failing_input)

Reducing inp...
Test #1 mystery(inp='V"/+!aF-(V4EOz*+s/Q,7)2@0_'): FAIL (ValueError: Invalid input)
Test #2 mystery(inp='z*+s/Q,7)2@0_'): PASS
Test #3 mystery(inp='V"/+!aF-(V4EO'): PASS
Test #4 mystery(inp='F-(V4EOz*+s/Q,7)2@0_'): FAIL (ValueError: Invalid input)
Test #5 mystery(inp='Oz*+s/Q,7)2@0_'): PASS
Test #6 mystery(inp='F-(V4E,7)2@0_'): FAIL (ValueError: Invalid input)
Test #7 mystery(inp=',7)2@0_'): PASS
Test #8 mystery(inp='F-(V4E'): PASS
Test #9 mystery(inp='V4E,7)2@0_'): PASS
Test #10 mystery(inp='F-(,7)2@0_'): FAIL (ValueError: Invalid input)
Test #11 mystery(inp=',7)2@0_'): PASS
Test #12 mystery(inp='F-(2@0_'): PASS
Test #13 mystery(inp='F-(,7)'): FAIL (ValueError: Invalid input)
Test #14 mystery(inp=',7)'): PASS
Test #15 mystery(inp='F-('): PASS
Test #16 mystery(inp='-(,7)'): FAIL (ValueError: Invalid input)
Test #17 mystery(inp='(,7)'): FAIL (ValueError: Invalid input)
Test #18 mystery(inp='7)'): PASS
Test #19 mystery(inp='(,'): PASS
Test #20 mystery(inp=',7)'): PASS
Tes

It is also possible to access the debugger programmatically:

In [52]:
dd = DeltaDebugger(show=False)
with dd:
    mystery(failing_input)

In [53]:
dd.args()

{'inp': 'V"/+!aF-(V4EOz*+s/Q,7)2@0_'}

In [54]:
dd.reduced_args()

{'inp': '()'}

For our ongoing `remove_html_markup()` example, we can reduce the failure-inducing input to a minimum, too:

In [55]:
from Assertions import remove_html_markup

In [56]:
with DeltaDebugger():
    remove_html_markup('"x > y"')

remove_html_markup(s='">')


If a function has multiple reducible variables, they get reduced in turns:

In [57]:
def string_error(s1, s2):
    assert s1 not in s2

In [58]:
with DeltaDebugger(log=True):
    string_error("foo", "foobar")

Reducing s1...
Test #1 string_error(s2='foobar', s1='foo'): FAIL (AssertionError)
Test #2 string_error(s2='foobar', s1='oo'): FAIL (AssertionError)
Test #3 string_error(s2='foobar', s1='o'): FAIL (AssertionError)
Reducing s2...
Test #1 string_error(s2='foobar', s1='o'): FAIL (AssertionError)
Test #2 string_error(s2='bar', s1='o'): PASS
Test #3 string_error(s2='foo', s1='o'): FAIL (AssertionError)
Test #4 string_error(s2='oo', s1='o'): FAIL (AssertionError)
Test #5 string_error(s2='o', s1='o'): FAIL (AssertionError)
Reducing s1...
Test #1 string_error(s2='o', s1='o'): FAIL (AssertionError)
Test #2 string_error(s2='o', s1='o'): FAIL (AssertionError)
string_error(s2='o', s1='o')


In [59]:
def list_error(l1, l2, maxlen):
    assert len(l1) < len(l2) < maxlen

In [60]:
with DeltaDebugger():
    list_error(l1=[1, 2, 3, 4, 5, 6, 7, 8, 9, 10], l2=[1, 2, 3], maxlen=5)

list_error(maxlen=5, l2=[3], l1=[10])


## Reducing Program Code

In [61]:
import inspect

One can also reduce _code_ in addition to input.

In [62]:
try:
    del remove_html_markup
except NameError:
    pass

In [63]:
import Assertions

In [64]:
assertions_source_code, _ = inspect.getsourcelines(Assertions)
assertions_source_code[:5]

['from bookutils import YouTubeVideo\n',
 'YouTubeVideo("9mI9sbKFkwU")\n',
 '\n',
 'import bookutils\n',
 '\n']

In [65]:
len(assertions_source_code)

426

In [66]:
def compile_and_run(lines):
    exec("".join(lines))

In [67]:
def compile_and_test_html_markup(lines):
    compile_and_run(lines + 
                    ['''\nassert remove_html_markup('"foo"') == '"foo"', "My Test"\n'''])

In [68]:
with ExpectError():
    compile_and_test_html_markup(assertions_source_code)

Traceback (most recent call last):
  File "<ipython-input-68-9a6cb59c37fe>", line 2, in <module>
    compile_and_test_html_markup(assertions_source_code)
  File "<ipython-input-67-4dd51a254e40>", line 3, in compile_and_test_html_markup
    ['''\nassert remove_html_markup('"foo"') == '"foo"', "My Test"\n'''])
  File "<ipython-input-66-c55288affc8c>", line 2, in compile_and_run
    exec("".join(lines))
  File "<string>", line 428, in <module>
AssertionError: My Test (expected)


In [69]:
dd = DeltaDebugger(log=False, show=False)
with dd:
    compile_and_test_html_markup(assertions_source_code)

In [70]:
len(dd.reduced_args()['lines'])

2

In [71]:
from bookutils import print_content

In [72]:
print_content("".join(dd.reduced_args()['lines']), ".py")

def remove_html_markup(s):
            tag = True

In [73]:
def compile_and_test_html_markup(lines):
    compile_and_run(lines + 
                    [
                        '''if remove_html_markup('<foo>bar</foo>') != 'bar':\n''',
                         '''    raise RuntimeError("Missing functionality")\n''',
                         '''assert remove_html_markup('"foo"') == '"foo"', "My Test"\n'''
                    ])

In [74]:
with ExpectError():
    compile_and_test_html_markup(dd.reduced_args()['lines'])

Traceback (most recent call last):
  File "<ipython-input-74-ffa6ade91b22>", line 2, in <module>
    compile_and_test_html_markup(dd.reduced_args()['lines'])
  File "<ipython-input-73-5872ba242ea4>", line 6, in compile_and_test_html_markup
    '''assert remove_html_markup('"foo"') == '"foo"', "My Test"\n'''
  File "<ipython-input-66-c55288affc8c>", line 2, in compile_and_run
    exec("".join(lines))
  File "<string>", line 4, in <module>
RuntimeError: Missing functionality (expected)


In [75]:
dd = DeltaDebugger(log=False, show=False)
with dd:
    compile_and_test_html_markup(assertions_source_code)

In [76]:
print_content("".join(dd.reduced_args()['lines']), ".py")

def remove_html_markup(s):
    tag = False
    quote = False
    out = ""
    for c in s:
        if c == '<' and not quote:
            tag = True
        elif c == '>' and not quote:
            tag = False
        elif c == '"' or c == "'" and tag:
            quote = not quote
        elif not tag:
            out = out + c
    return out

This is the same function as in the [chapter on assertions](Assertions.ipynb). However, the reduced version shows small differences compared to the original:

In [77]:
remove_html_markup_source_code, _ = inspect.getsourcelines(Assertions.remove_html_markup)
print_content("".join(remove_html_markup_source_code), ".py")

def remove_html_markup(s):
    tag = False
    quote = False
    out = ""

    for c in s:
        if c == '<' and not quote:
            tag = True
        elif c == '>' and not quote:
            tag = False
        elif c == '"' or c == "'" and tag:
            quote = not quote
        elif not tag:
            out = out + c

    # postcondition
    assert '<' not in out and '>' not in out

    return out

In [78]:
quiz("In the reduced version, what has changed?",
    [
        "Comments are deleted",
        "Blank lines are deleted",
        "Initializations are deleted",
        "The assertion is deleted",
    ], [2 ** n for n in range(0, 3)]
    )

## Synopsis

A _reducer_ takes a failure-inducing input and reduces it to the minimum that still reproduces the failure.  This chapter provides a `DeltaDebugger` class that implements such a reducer.

Here is a simple example: An arithmetic expression causes an error in the Python interpreter:

In [79]:
def myeval(inp):
    return eval(inp)

In [80]:
with ExpectError():
    myeval('1 + 2 * 3 / 0')

Traceback (most recent call last):
  File "<ipython-input-80-deae668eac94>", line 2, in <module>
    myeval('1 + 2 * 3 / 0')
  File "<ipython-input-79-e6018665ac61>", line 2, in myeval
    return eval(inp)
  File "<string>", line 1, in <module>
ZeroDivisionError: division by zero (expected)


Can we reduce this input to a minimum? _Delta Debugging_ is a simple and robust reduction algorithm. We provide a `DeltaDebugger` class that is used in conjunction with a (failing) function call:

```python
with DeltaDebugger():
    fun(args...)
```

The class automatically determines minimal arguments that cause the function to fail with the same exception as the original. By default, these minimal arguments are simply printed out:

In [81]:
with DeltaDebugger():
    myeval('1 + 2 * 3 / 0')

myeval(inp='2/0')


The input is reduced to the maximum: We get the essence of the division by zero.

There also is an interface to access the reduced input(s) programmatically:

In [82]:
dd = DeltaDebugger(show=False)
with dd:
    myeval('1 + 2 * 3 / 0')
dd.reduced_args()

{'inp': '2/0'}

`DelteDebugger` works on all functions that take a collection (a string, a list, a set) as one or more of their inputs.

## Lessons Learned

* Reducing failure-inducing inputs to a minimum is helpful for testing and debugging.
* _Delta debugging_ is a simple and robust algorithm to easily reduce test cases.
* One can also apply delta debugging to _code_, yielding minimal code that still produces an error.
* Precisely specifying failure conditions helps avoiding false diagnoses.

## Next Steps

Our next chapter focuses on finding failure-inducing code changes.

## Background

The "lexical" delta debugging algorithm discussed here stems from \cite{Zeller2002}; actually, this is the exact Python implementation as used by Zeller in 2002.  The idea of systematically reducing inputs has been discovered a number of times, although not as  automatic and generic as delta debugging. \cite{Slutz1998}, for instance, discusses systematic reduction of SQL statements for SQL databases; the general process as manual work is well described by \cite{Kernighan1999}.

The deficits of delta debugging as it comes to syntactically complex inputs were first discussed in *compiler testing*, and _reducing tree inputs_ rather than string inputs was quickly discovered as an alternative.  *Hierarchical Delta Debugging* (*HDD*) \cite{Misherghi2006} applies delta debugging on subtrees of a parse tree, systematically reducing a parse tree to a minimum.  _Generalized Tree Reduction_ \cite{Herfert2017} generalizes this idea to apply arbitrary _patterns_ such as replacing a term by a compatible term in a subtree.  Using _grammars_ to reduce inputs was first implemented in the _Perses_ tool \cite{Sun2018}.

While applying delta debugging to code lines does a decent job, _syntactic_ and especially _language-specific_ approaches can do a much better job for the programming language at hand.  *C-Reduce* \cite{Regehr2012} is a reducer specifically targeting the reduction of programming languages.  Besides reductions in the style of delta debugging or tree transformations, C-Reduce comes with more than 30 source-to-source transformations that replace aggregates by scalars, remove function parameters at a definition and all call sites, change functions to return `void` and deleting all `return` statements, and many more.  While specifically instantiated for the C language (and used for testing C compilers), these principles extend to arbitrary programming languages following an ALGOL-like syntax.  When testing a compiler, C-Reduce is the tool to go for.

This [blog post](https://www.drmaciver.com/2019/01/notes-on-test-case-reduction/) by David McIver contains lots of insights on how to apply reduction in practice, in particular multiple runs with different abstraction levels.

## Exercises

How to best reduce inputs is still an underdeveloped field of research, with lots of opportunities.

### Exercise 1: Syntactic Code Reduction

When reducing program code, using a _syntactic_ approach is a much better alternative to the _line-by-line_ approach discussed above.